In [3]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [5]:
tf.keras.backend.clear_session()

In [6]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [9]:
subject = 'Goiás - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [10]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_GO.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Ano,Goiás - IDH,Goiás - IDH Longevidade,Goiás - IDH Renda,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - IDH Educacao,Goiás - Produção de Cimento (t),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Goiás - Consumo de Cimento (t)
0,2008,0.750753,0.786865,0.740091,9.979485e+07,5.727076e+06,14.203138,8.592012e+07,0.726402,1049.218132,...,2.868108,11.760983,25.567101,931.224932,1376.621797,23.533988,16.021792,67.029130,1.258587,2310.616000
1,2009,0.751479,0.787352,0.738646,1.005138e+08,5.757645e+06,14.197250,8.596210e+07,0.729972,1167.838500,...,2.305885,9.472585,26.358065,931.509364,1404.486199,23.539793,15.895700,66.852293,1.302609,2335.717000
2,2010,0.752088,0.787273,0.737500,1.030813e+08,5.806966e+06,14.241313,8.663480e+07,0.733384,1306.390438,...,1.958246,9.332485,26.324590,931.774922,1422.397943,23.407790,15.594180,66.528157,1.346857,2657.527000
3,2011,0.752581,0.786636,0.736643,1.054535e+08,5.798734e+06,14.278825,8.718791e+07,0.736630,1430.447037,...,2.079088,10.985689,26.891246,932.026640,1424.732773,23.379311,15.713953,67.017525,1.343835,2901.707000
4,2012,0.752723,0.785429,0.735750,1.075784e+08,5.775231e+06,14.289292,8.762067e+07,0.739409,1500.429389,...,2.183282,8.159014,28.111519,932.153867,1426.391027,23.445549,15.826075,67.022758,1.300550,3061.533000
5,2013,0.753450,0.783632,0.736229,1.092519e+08,5.758466e+06,14.261467,8.779968e+07,0.742860,1468.429450,...,2.403445,7.913561,27.073883,932.317691,1424.468340,23.563897,15.775697,66.406360,1.252025,3181.778000
6,2014,0.753571,0.781434,0.735915,1.105942e+08,5.711944e+06,14.238295,8.787916e+07,0.745598,1413.139667,...,2.748397,10.350442,25.841819,932.468684,1423.836782,23.878739,15.683323,65.088818,1.241766,2861.897429
7,2015,0.753286,0.778804,0.735658,1.115113e+08,5.628820e+06,14.206629,8.784849e+07,0.747481,1402.398186,...,2.501529,12.454398,25.709892,932.508663,1430.027526,24.014961,15.540525,64.054609,1.245195,2517.290743
8,2016,0.756276,0.784027,0.728777,1.120977e+08,5.573901e+06,14.184179,8.781949e+07,0.758668,1460.206906,...,2.073464,13.111908,26.601698,932.486431,1433.688880,23.900472,15.251199,63.441573,1.236011,2193.545829
9,2017,0.755573,0.780939,0.729212,1.123228e+08,5.555871e+06,14.177721,8.783412e+07,0.759092,1589.529694,...,2.177386,9.479812,26.603489,932.515580,1426.852240,23.876158,15.213263,63.423776,1.188108,2270.346000


In [11]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Goiás - IDH,Goiás - IDH Longevidade,Goiás - IDH Renda,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - IDH Educacao,Goiás - Produção de Cimento (t),Goiás - Desemprego,...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,0.750753,0.786865,0.740091,9.979485e+07,5.727076e+06,14.203138,8.592012e+07,0.726402,1049.218132,7.941146,...,-7723.574973,2.868108,11.760983,25.567101,931.224932,1376.621797,23.533988,16.021792,67.029130,1.258587
1,0.751479,0.787352,0.738646,1.005138e+08,5.757645e+06,14.197250,8.596210e+07,0.729972,1167.838500,7.969157,...,-8961.781709,2.305885,9.472585,26.358065,931.509364,1404.486199,23.539793,15.895700,66.852293,1.302609
2,0.752088,0.787273,0.737500,1.030813e+08,5.806966e+06,14.241313,8.663480e+07,0.733384,1306.390438,7.932921,...,-9895.682385,1.958246,9.332485,26.324590,931.774922,1422.397943,23.407790,15.594180,66.528157,1.346857
3,0.752581,0.786636,0.736643,1.054535e+08,5.798734e+06,14.278825,8.718791e+07,0.736630,1430.447037,7.921619,...,-9989.170714,2.079088,10.985689,26.891246,932.026640,1424.732773,23.379311,15.713953,67.017525,1.343835
4,0.752723,0.785429,0.735750,1.075784e+08,5.775231e+06,14.289292,8.762067e+07,0.739409,1500.429389,7.786290,...,-9429.326328,2.183282,8.159014,28.111519,932.153867,1426.391027,23.445549,15.826075,67.022758,1.300550
5,0.753450,0.783632,0.736229,1.092519e+08,5.758466e+06,14.261467,8.779968e+07,0.742860,1468.429450,8.389968,...,-7157.441850,2.403445,7.913561,27.073883,932.317691,1424.468340,23.563897,15.775697,66.406360,1.252025
6,0.753571,0.781434,0.735915,1.105942e+08,5.711944e+06,14.238295,8.787916e+07,0.745598,1413.139667,9.015756,...,-3453.461327,2.748397,10.350442,25.841819,932.468684,1423.836782,23.878739,15.683323,65.088818,1.241766
7,0.753286,0.778804,0.735658,1.115113e+08,5.628820e+06,14.206629,8.784849e+07,0.747481,1402.398186,9.398111,...,-1442.919387,2.501529,12.454398,25.709892,932.508663,1430.027526,24.014961,15.540525,64.054609,1.245195
8,0.756276,0.784027,0.728777,1.120977e+08,5.573901e+06,14.184179,8.781949e+07,0.758668,1460.206906,9.378450,...,-3128.377228,2.073464,13.111908,26.601698,932.486431,1433.688880,23.900472,15.251199,63.441573,1.236011
9,0.755573,0.780939,0.729212,1.123228e+08,5.555871e+06,14.177721,8.783412e+07,0.759092,1589.529694,8.980786,...,-5386.337108,2.177386,9.479812,26.603489,932.515580,1426.852240,23.876158,15.213263,63.423776,1.188108


In [12]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     2335.717000
1     2657.527000
2     2901.707000
3     3061.533000
4     3181.778000
5     2861.897429
6     2517.290743
7     2193.545829
8     2270.346000
9     2282.903000
10    2367.716000
11    2438.585000
12    2782.135000
13    2807.973000
14            NaN
Name: Goiás - Consumo de Cimento (t), dtype: float64

In [13]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Goiás - IDH,Goiás - IDH Longevidade,Goiás - IDH Renda,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - IDH Educacao,Goiás - Produção de Cimento (t),Goiás - Desemprego,...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.667342,0.949720,1.410033,-1.833449,0.433923,-0.325155,-1.986396,-1.569745,-1.852836,-1.027093,...,-0.569731,2.026352,0.984445,-1.331393,-2.215071,-2.920210,-0.729782,1.542463,1.077765,0.191401
1,-1.194211,1.061657,1.060251,-1.672467,0.753723,-0.464589,-1.926962,-1.269707,-1.306173,-0.979720,...,-0.838980,0.085558,-0.051710,-0.314120,-1.543139,-1.011487,-0.705275,1.108832,0.970139,0.769359
2,-0.797383,1.043475,0.782547,-1.097605,1.269707,0.578899,-0.974517,-0.982875,-0.667656,-1.041004,...,-1.042057,-1.114490,-0.115145,-0.357173,-0.915796,0.215476,-1.262514,0.071898,0.772865,1.350269
3,-0.475496,0.897221,0.575051,-0.566452,1.183590,1.467272,-0.191390,-0.710011,-0.095941,-1.060119,...,-1.062386,-0.697344,0.633403,0.371613,-0.321146,0.375413,-1.382735,0.483799,1.070702,1.310592
4,-0.382925,0.620129,0.358866,-0.090684,0.937709,1.715155,0.421338,-0.476466,0.226574,-1.288995,...,-0.940648,-0.337667,-0.646476,1.941026,-0.020591,0.489004,-1.103118,0.869388,1.073887,0.742316
5,0.090857,0.207420,0.474906,0.284004,0.762311,1.056209,0.674789,-0.186418,0.079102,-0.268017,...,-0.446625,0.422337,-0.757614,0.606505,0.366422,0.357299,-0.603521,0.696138,0.698737,0.105254
6,0.169632,-0.297081,0.398809,0.584563,0.275613,0.507437,0.787318,0.043747,-0.175702,0.790354,...,0.358807,1.613108,0.345772,-0.978073,0.723121,0.314037,0.725548,0.378463,-0.103139,-0.029436
7,-0.016584,-0.901022,0.336552,0.789891,-0.594012,-0.242472,0.743896,0.202034,-0.225204,1.437016,...,0.796000,0.760923,1.298413,-1.147746,0.817566,0.738105,1.300597,-0.112621,-0.732575,0.015582
8,1.932066,0.298219,-1.329669,0.921200,-1.168560,-0.774142,0.702846,1.142334,0.041208,1.403764,...,0.429496,-0.716758,1.596124,-0.000780,0.765046,0.988910,0.817294,-1.107619,-1.105679,-0.104987
9,1.474402,-0.410869,-1.224492,0.971602,-1.357181,-0.927081,0.723547,1.178003,0.637193,0.731211,...,-0.061498,-0.358019,-0.048438,0.001524,0.833907,0.520597,0.714653,-1.238081,-1.116510,-0.733882


In [14]:
reshaped_train_input = dfToInputRNN(train_input)

In [15]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 0.048551  , -1.49823661, -1.31974416,  0.77589662,
         -1.06999983, -1.13620542,  0.46780016,  1.21002656,
          1.5249394 ,  0.67235226, -1.0453709 ,  1.21859064,
         -0.99976018,  0.79428371,  1.15575749, -0.52559858,
          1.30260559, -0.55253046, -0.25130359,  1.38787176,
          1.47154451, -1.80649616,  1.09296291,  0.94281138,
          0.52836084,  0.61258103,  0.87656257,  1.47801536,
          1.71982086,  1.28221181,  1.39086472,  1.58209844,
          1.08872091,  0.55212962, -1.81398118,  1.44113674,
          1.11191094,  1.30470835,  0.86390516, -1.67365968,
         -1.70738939, -1.55813196, -0.48498301, -0.75714639,
          0.40770893, -0.27394734,  2.02323219,  1.2908951 ,
          1.70465513,  1.81148084,  1.39968653, -1.36191442,
          1.28024593,  1.11054795,  0.12662799,  1.40414112,
          1.40468999,  1.45643959,  1.58509336,  1.32283157,
          1.30222047,  1.55138602,  1.91801602, -0.77489784,
          2.32098884, -1

In [16]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     2335.717000
1     2657.527000
2     2901.707000
3     3061.533000
4     3181.778000
5     2861.897429
6     2517.290743
7     2193.545829
8     2270.346000
9     2282.903000
10    2367.716000
11    2438.585000
Name: Goiás - Consumo de Cimento (t), dtype: float64

In [17]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    2438.585
Name: Goiás - Consumo de Cimento (t), dtype: float64

In [18]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Goiás - IDH,Goiás - IDH Longevidade,Goiás - IDH Renda,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - IDH Educacao,Goiás - Produção de Cimento (t),Goiás - Desemprego,...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,-1.158228,-2.357097,-1.338521,0.495067,-0.996697,-1.327004,0.145995,1.141532,2.088469,0.318701,...,0.964765,-1.482779,-2.260447,0.946592,0.252129,0.101107,0.76294,-1.953922,-1.523095,-2.072592


In [19]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[-1.15822784, -2.35709698, -1.33852115,  0.49506741,
         -0.99669724, -1.32700383,  0.14599457,  1.14153249,
          2.08846893,  0.3187012 ,  1.10840491,  2.63582309,
         -0.18995911,  0.72640489,  2.66833308, -0.032632  ,
          0.98802435,  0.24980494, -0.08955808,  1.85434394,
          2.02706568, -2.21150703,  1.42938591,  0.85132668,
          0.14092386,  0.30902324,  1.00267904,  2.27970137,
          1.11272593,  0.61007902,  0.89064538,  1.42231438,
          1.02650562,  1.47127593, -1.53201473,  1.73503858,
          1.66610613,  1.0456894 ,  0.55289259, -1.81216593,
         -2.10342296, -2.55377478,  0.41940628, -1.75756445,
          0.241023  , -0.98659345,  1.52184916,  1.01398607,
          1.49290967,  1.40764092,  1.87930411, -2.51892072,
          2.83356214,  1.58620924, -0.02524128,  1.54231542,
          1.5176119 ,  1.7355269 ,  1.87941982,  1.54198129,
          1.51276712,  1.83795594,  2.62210693,  1.04993174,
          0.96476493, -1

In [20]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    2782.135
Name: Goiás - Consumo de Cimento (t), dtype: float64

In [21]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh')),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        batch_size=train_input.shape[0],
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [22]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [24]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[45312686, 3284444844, 1855006969, 2253735557, 1256099862, 2489422914, 1591407551, 3369405354, 3034951144, 2827725421, 917044163, 1062718162, 1911418798, 391342687, 2510183572, 3623180707, 1542183000, 243896409, 2529388612, 943593794, 4007114048, 596271866, 4136997820, 359973484, 3695595523, 233196867, 2157380285, 4294569123, 2674201586, 1558630400, 766464380, 3386164373, 3918337061, 3067332545, 3165248066, 213268733, 4119620279, 3869179232, 4097354166, 3876876464, 1273904335, 2719669168, 1868859595, 3274117766, 1585998555, 362951501, 3562213799, 1262606673, 3320862406, 1110812555, 1962905977, 2295522102, 1449571603, 2524564670, 4166552565, 2040365162, 434052997, 1697400337, 3529892293, 451496193, 3313388374, 3785904869, 3930323512, 1439330270, 999860789, 3662650210, 578777284, 3094883131, 2777678071, 4081964245, 3084699117, 3847966277, 680456358, 4012556905, 442963902, 2957471956, 2262471029, 4156262172, 2244507321, 4135507166, 1914907642, 22182130, 1837117265, 1900607246, 3786430531,

loss: 18732.837890625


Step: 91 ___________________________________________
loss: 14745.6298828125


Step: 92 ___________________________________________
loss: 9165.5771484375


Step: 93 ___________________________________________
loss: 14645.1748046875


Step: 94 ___________________________________________
loss: 14209.0419921875


Step: 95 ___________________________________________
loss: 11968.8271484375


Step: 96 ___________________________________________
loss: 17275.0625


Step: 97 ___________________________________________
loss: 15346.6611328125


Step: 98 ___________________________________________
loss: 16303.3203125


Step: 99 ___________________________________________
loss: 12500.2685546875


final_seed: 4097354166


In [25]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 2s 2s/step - loss: 6807647.5000
Epoch 2/10000
1/1 [==============================] - 0s 4ms/step - loss: 6509916.0000
Epoch 3/10000
1/1 [==============================] - 0s 3ms/step - loss: 2234840.2500
Epoch 4/10000
1/1 [==============================] - 0s 3ms/step - loss: 1103158.1250
Epoch 5/10000
1/1 [==============================] - 0s 3ms/step - loss: 2248583.5000
Epoch 6/10000
1/1 [==============================] - 0s 4ms/step - loss: 757475.8125
Epoch 7/10000
1/1 [==============================] - 0s 4ms/step - loss: 224630.8125
Epoch 8/10000
1/1 [==============================] - 0s 4ms/step - loss: 180135.0781
Epoch 9/10000
1/1 [==============================] - 0s 4ms/step - loss: 74921.4453
Epoch 10/10000
1/1 [==============================] - 0s 4ms/step - loss: 99390.7734
Epoch 11/10000
1/1 [==============================] - 0s 4ms/step - loss: 81583.4688
Epoch 12/10000
1/1 [==============================] - 0s 4ms/s

1/1 [==============================] - 0s 4ms/step - loss: 66693.6172
Epoch 98/10000
1/1 [==============================] - 0s 4ms/step - loss: 47096.0000
Epoch 99/10000
1/1 [==============================] - 0s 4ms/step - loss: 45250.1719
Epoch 100/10000
1/1 [==============================] - 0s 5ms/step - loss: 50678.3711
Epoch 101/10000
1/1 [==============================] - 0s 4ms/step - loss: 70491.8828
Epoch 102/10000
1/1 [==============================] - 0s 4ms/step - loss: 46474.5898
Epoch 103/10000
1/1 [==============================] - 0s 4ms/step - loss: 68848.3750
Epoch 104/10000
1/1 [==============================] - 0s 4ms/step - loss: 116041.3516
Epoch 105/10000
1/1 [==============================] - 0s 4ms/step - loss: 48164.4102
Epoch 106/10000
1/1 [==============================] - 0s 4ms/step - loss: 51806.7695
Epoch 107/10000
1/1 [==============================] - 0s 4ms/step - loss: 77184.0781
Epoch 108/10000
1/1 [==============================] - 0s 4ms/step - lo

1/1 [==============================] - 0s 4ms/step - loss: 55111.9375
Epoch 193/10000
1/1 [==============================] - 0s 3ms/step - loss: 65210.2227
Epoch 194/10000
1/1 [==============================] - 0s 4ms/step - loss: 65801.8750
Epoch 195/10000
1/1 [==============================] - 0s 4ms/step - loss: 56883.2344
Epoch 196/10000
1/1 [==============================] - 0s 4ms/step - loss: 61518.2812
Epoch 197/10000
1/1 [==============================] - 0s 4ms/step - loss: 59387.5195
Epoch 198/10000
1/1 [==============================] - 0s 4ms/step - loss: 55360.1367
Epoch 199/10000
1/1 [==============================] - 0s 4ms/step - loss: 49875.2695
Epoch 200/10000
1/1 [==============================] - 0s 4ms/step - loss: 29312.4121
Epoch 201/10000
1/1 [==============================] - 0s 4ms/step - loss: 73807.7422
Epoch 202/10000
1/1 [==============================] - 0s 4ms/step - loss: 63030.4531
Epoch 203/10000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 29144.9688
Epoch 288/10000
1/1 [==============================] - 0s 4ms/step - loss: 47438.3008
Epoch 289/10000
1/1 [==============================] - 0s 4ms/step - loss: 61862.9688
Epoch 290/10000
1/1 [==============================] - 0s 4ms/step - loss: 35670.9648
Epoch 291/10000
1/1 [==============================] - 0s 4ms/step - loss: 90385.3984
Epoch 292/10000
1/1 [==============================] - 0s 3ms/step - loss: 48052.0000
Epoch 293/10000
1/1 [==============================] - 0s 4ms/step - loss: 34811.2227
Epoch 294/10000
1/1 [==============================] - 0s 4ms/step - loss: 44106.5938
Epoch 295/10000
1/1 [==============================] - 0s 3ms/step - loss: 88006.2188
Epoch 296/10000
1/1 [==============================] - 0s 4ms/step - loss: 61189.3945
Epoch 297/10000
1/1 [==============================] - 0s 4ms/step - loss: 71319.3750
Epoch 298/10000
1/1 [==============================] - 0s 3ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 80638.6172
Epoch 383/10000
1/1 [==============================] - 0s 4ms/step - loss: 43033.7969
Epoch 384/10000
1/1 [==============================] - 0s 4ms/step - loss: 46248.6055
Epoch 385/10000
1/1 [==============================] - 0s 4ms/step - loss: 32801.4336
Epoch 386/10000
1/1 [==============================] - 0s 4ms/step - loss: 56021.1133
Epoch 387/10000
1/1 [==============================] - 0s 4ms/step - loss: 42960.9727
Epoch 388/10000
1/1 [==============================] - 0s 4ms/step - loss: 78026.1562
Epoch 389/10000
1/1 [==============================] - 0s 4ms/step - loss: 47118.4492
Epoch 390/10000
1/1 [==============================] - 0s 4ms/step - loss: 66673.4297
Epoch 391/10000
1/1 [==============================] - 0s 4ms/step - loss: 67291.7109
Epoch 392/10000
1/1 [==============================] - 0s 4ms/step - loss: 42577.0195
Epoch 393/10000
1/1 [==============================] - 0s 4ms/step - l

...

1/1 [==============================] - 0s 4ms/step - loss: 55488.7812
Epoch 668/10000
1/1 [==============================] - 0s 4ms/step - loss: 38468.1914
Epoch 669/10000
1/1 [==============================] - 0s 3ms/step - loss: 70239.2891
Epoch 670/10000
1/1 [==============================] - 0s 4ms/step - loss: 51763.0156
Epoch 671/10000
1/1 [==============================] - 0s 4ms/step - loss: 42849.0586
Epoch 672/10000
1/1 [==============================] - 0s 4ms/step - loss: 60393.3320
Epoch 673/10000
1/1 [==============================] - 0s 3ms/step - loss: 35434.7773
Epoch 674/10000
1/1 [==============================] - 0s 4ms/step - loss: 35310.4375
Epoch 675/10000
1/1 [==============================] - 0s 4ms/step - loss: 110915.3828
Epoch 676/10000
1/1 [==============================] - 0s 4ms/step - loss: 72951.8047
Epoch 677/10000
1/1 [==============================] - 0s 5ms/step - loss: 45987.6289
Epoch 678/10000
1/1 [==============================] - 0s 4ms/step - 

1/1 [==============================] - 0s 4ms/step - loss: 66927.9922
Epoch 763/10000
1/1 [==============================] - 0s 4ms/step - loss: 68267.7266
Epoch 764/10000
1/1 [==============================] - 0s 4ms/step - loss: 46856.5508
Epoch 765/10000
1/1 [==============================] - 0s 4ms/step - loss: 128434.7891
Epoch 766/10000
1/1 [==============================] - 0s 4ms/step - loss: 55943.4688
Epoch 767/10000
1/1 [==============================] - 0s 4ms/step - loss: 72966.0625
Epoch 768/10000
1/1 [==============================] - 0s 4ms/step - loss: 58725.3633
Epoch 769/10000
1/1 [==============================] - 0s 4ms/step - loss: 54832.9844
Epoch 770/10000
1/1 [==============================] - 0s 4ms/step - loss: 87827.8672
Epoch 771/10000
1/1 [==============================] - 0s 4ms/step - loss: 53840.9180
Epoch 772/10000
1/1 [==============================] - 0s 3ms/step - loss: 69983.6328
Epoch 773/10000
1/1 [==============================] - 0s 3ms/step - 

1/1 [==============================] - 0s 4ms/step - loss: 59480.1367
Epoch 858/10000
1/1 [==============================] - 0s 4ms/step - loss: 101582.4453
Epoch 859/10000
1/1 [==============================] - 0s 4ms/step - loss: 33567.1289
Epoch 860/10000
1/1 [==============================] - 0s 4ms/step - loss: 53573.7695
Epoch 861/10000
1/1 [==============================] - 0s 4ms/step - loss: 93371.0625
Epoch 862/10000
1/1 [==============================] - 0s 4ms/step - loss: 39826.3438
Epoch 863/10000
1/1 [==============================] - 0s 4ms/step - loss: 44378.3711
Epoch 864/10000
1/1 [==============================] - 0s 4ms/step - loss: 68338.3359
Epoch 865/10000
1/1 [==============================] - 0s 4ms/step - loss: 35475.8633
Epoch 866/10000
1/1 [==============================] - 0s 4ms/step - loss: 42822.1797
Epoch 867/10000
1/1 [==============================] - 0s 4ms/step - loss: 47073.3867
Epoch 868/10000
1/1 [==============================] - 0s 4ms/step - 

1/1 [==============================] - 0s 4ms/step - loss: 63667.0312
Epoch 953/10000
1/1 [==============================] - 0s 4ms/step - loss: 36586.2031
Epoch 954/10000
1/1 [==============================] - 0s 4ms/step - loss: 50575.9180
Epoch 955/10000
1/1 [==============================] - 0s 4ms/step - loss: 87781.7891
Epoch 956/10000
1/1 [==============================] - 0s 4ms/step - loss: 42716.0352
Epoch 957/10000
1/1 [==============================] - 0s 4ms/step - loss: 74102.5078
Epoch 958/10000
1/1 [==============================] - 0s 4ms/step - loss: 39093.2500
Epoch 959/10000
1/1 [==============================] - 0s 4ms/step - loss: 53287.0898
Epoch 960/10000
1/1 [==============================] - 0s 4ms/step - loss: 26943.8496
Epoch 961/10000
1/1 [==============================] - 0s 4ms/step - loss: 36784.6758
Epoch 962/10000
1/1 [==============================] - 0s 4ms/step - loss: 29373.4375
Epoch 963/10000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 42391.9023
Epoch 1047/10000
1/1 [==============================] - 0s 4ms/step - loss: 46803.2227
Epoch 1048/10000
1/1 [==============================] - 0s 5ms/step - loss: 69749.5312
Epoch 1049/10000
1/1 [==============================] - 0s 4ms/step - loss: 59239.2656
Epoch 1050/10000
1/1 [==============================] - 0s 4ms/step - loss: 37908.6914
Epoch 1051/10000
1/1 [==============================] - 0s 4ms/step - loss: 85942.4688
Epoch 1052/10000
1/1 [==============================] - 0s 4ms/step - loss: 52335.5938
Epoch 1053/10000
1/1 [==============================] - 0s 4ms/step - loss: 26813.3887
Epoch 1054/10000
1/1 [==============================] - 0s 4ms/step - loss: 53824.8867
Epoch 1055/10000
1/1 [==============================] - 0s 5ms/step - loss: 74068.0703
Epoch 1056/10000
1/1 [==============================] - 0s 5ms/step - loss: 53053.6992
Epoch 1057/10000
1/1 [==============================] - 0s 5

In [26]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 577ms/step
[2782.135](test_target) - [2401.9436](prediction) = 380.1913964843752


In [27]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.13665454641287184

In [28]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [29]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Goiás - IDH,Goiás - IDH Longevidade,Goiás - IDH Renda,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - IDH Educacao,Goiás - Produção de Cimento (t),Goiás - Desemprego,...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.258939,-1.397925,1.268852,-0.94647,-1.118690,-0.551011,-0.770233,-1.233828,-1.191912,-0.425199,...,1.277766,1.308949,1.412346,-1.413223,-1.238512,-1.303278,0.658911,1.028795,1.089129,-1.223704
1,0.071523,0.884312,-0.093583,-0.43679,-0.189921,-0.852453,-0.642042,0.018373,-0.063218,1.380677,...,-0.114016,-0.190804,-0.643250,0.752439,0.028014,0.176141,0.754232,0.325951,0.236684,-0.002080
2,1.187416,0.513613,-1.175269,1.38326,1.308611,1.403464,1.412276,1.215455,1.255129,-0.955478,...,-1.163750,-1.118144,-0.769096,0.660784,1.210497,1.127138,-1.413142,-1.354746,-1.325813,1.225784


train_target:


,Goiás - Consumo de Cimento (t)
0,2335.717
1,2657.527
2,2901.707


test_input:


...

,Goiás - IDH,Goiás - IDH Longevidade,Goiás - IDH Renda,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - IDH Educacao,Goiás - Produção de Cimento (t),Goiás - Desemprego,...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,-1.158228,-2.357097,-1.338521,0.495067,-0.996697,-1.327004,0.145995,1.141532,2.088469,0.318701,...,0.964765,-1.482779,-2.260447,0.946592,0.252129,0.101107,0.76294,-1.953922,-1.523095,-2.072592


test_target:


,Goiás - Consumo de Cimento (t)
12,2782.135


1/1 [==============================] - 1s 551ms/step
Error: 380.1913964843752




[2853.438720703125,
 2949.8564453125,
 2841.9736328125,
 2785.879638671875,
 2424.453369140625,
 2348.34423828125,
 2241.660888671875,
 2247.739501953125,
 2426.645263671875,
 2401.943603515625]

In [30]:
display(targets)
display(predictions)

[3061.533,
 3181.778,
 2861.8974285714285,
 2517.2907428571425,
 2193.5458285714285,
 2270.346,
 2282.903,
 2367.716,
 2438.585,
 2782.135]

[2853.438720703125,
 2949.8564453125,
 2841.9736328125,
 2785.879638671875,
 2424.453369140625,
 2348.34423828125,
 2241.660888671875,
 2247.739501953125,
 2426.645263671875,
 2401.943603515625]

In [31]:
mae = mean_absolute_error(predictions, targets)
mae

159.07840465959822

In [32]:
porcentage = mae/np.mean(targets)
porcentage

0.06128363484002578